# Manual for pre-processing local data for running MTUQ

#### Félix Rodríguez-Cardozo and Jochen Braunmiller

This manual is intended for reading SAC files to pre-process them for creating SAC files in the format required by [MTUQ](https://github.com/uafgeotools/mtuq) for running a moment tensor grid-search. SAC file name requirement is to end in '.sac' 

Before starting, let's explore the directory to become familiar with the minimun requirements to use this notebook. 

#### Note:  If you want to run  this notebook again clean the workspace by calling the subroutine clean()

In [ ]:
import os
def clean():
    print('rm -r PROCESSED_DATA BU_PROCESSED_DATA RESP_FILES')
    os.system('rm -r PROCESSED_DATA BU_PROCESSED_DATA RESP_FILES __pycache__')

In [ ]:
#If you want to clean the workspace run this block of code
clean()

In [ ]:
#ls for seeing the files in the preprocessing directory
import os
! ls -d *

# Directory Overview - Getting Started 

### 1. **DATA** 
This directory contains the files in SAC format that will be processed. 

In [ ]:
#ls DATA/* for seeing the raw SAC files 
!ls DATA/20171201023244/*

For this example we provide data from 3 seismic stations in Iran. The data come from the Iranian National Seismological Network (ISNS) and the Iranian Seismological Center (IRSC). The way in which the SAC files are named is slightly different between the two data sources. Still, as long as the file names are ending in '.sac' this notebook should be able to work with any name. It is propably a good idea that file names say something about station name, channel, (network), and date-and-time of data (perhaps when they start). 

The directory with the SAC files for each event should follow the format: **yyyymmddhhiiss**. Where **yyyy** stands for year, **mm** for month, **dd** for day, **hh** for hour, **ii** for minute and **ss** for seconds. 

#### NOTE: It is important that the SAC files have a header with, at minimum information, channel and station name.

In [ ]:
#Let's see the header information of one of the seismograms:
from obspy import read
st = read('DATA/20171201023244/171201.0232.BSRN.BHZ.sac')
print(st[0].stats)

### 2. events_list.txt 

Plain text file with basic information about origin time, hypocenter, and magnitude of the to-be-pre-processed earthquake. The SAC files corresponding to the events listed in events_list.txt should be saved in the **DATA** directory following the aforementioned format.

The events list has the following format:

**YEAR MONTH DAY HOUR:MIN:SEC LATITUDE LONGITUDE MAGNITUDE DEPTH**

In [ ]:
#Let's see the events_list.txt file
! cat events_list.txt

### 3. pzfiles 

Directory with the SAC pole and zero files needed to remove the instrument response. The pole-and-zero SAC file names have to follow the format **STATIONNAME_CHANNEL.pz**

In [ ]:
#Let's see the content of one the poles and zeros files
! ls pzfiles
! cat pzfiles/BSRN_BHZ.pz

#### 3. station_list.txt:

Plain text file with seismic station information for stations with data to be pre-processed. This file should follow the format:

**STATIONNAME LATITUDE LONGITUDE ELEVATION_IN_METERS NETWORK**

Note: Elevation is not used further but an entry in this column is required.

In [ ]:
# Let's see the station_list.txt content
! cat station_list.txt

### 4. run_MTUQ: 

Once the seismograms are preprocessed, this directory contains an example for running MTUQ. We will see details of that directory and how to run MTUQ later.

### 5. local_preprocessing_mtuq.py:

Python script that is the module for preprocessing the SAC files.

# Preprocessing SAC files

To use this notebook it is important to have already loaded the MTUQ module. In addition, it is important to load the following libraries including the one designed for preprocessing the local data (**local_preprocessing_mtuq.py**). 

In [ ]:
#Loading libraries to use in this notebook
import local_preprocessing_mtuq
import importlib
importlib.reload(local_preprocessing_mtuq)
from obspy import read
import matplotlib.pyplot as plt
import io
from obspy import Trace
from obspy.io.sac import sacpz
from obspy import read, read_inventory

Run the follow block of code. It is a function for plotting seismograms and we will use it later. 

In [ ]:
#Function for comparing two seismograms
def plot_event(tr1,tr2,zoom):
    
    lim1 = round(len(tr1.times("matplotlib"))*zoom[0])
    lim2 = round(len(tr1.times("matplotlib"))*zoom[1])
    
    x1 = tr1.times("matplotlib")[lim1:len(tr1.times("matplotlib"))-lim2]
    y1 = tr1.data[lim1:len(tr1.times("matplotlib"))-lim2]
    
    x2 = tr2.times("matplotlib")[lim1:len(tr1.times("matplotlib"))-lim2]
    y2 = tr2.data[lim1:len(tr1.times("matplotlib"))-lim2]
    
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    
    ax.plot(x1, y1, "b-", label='Raw data')
    ax.plot(x2, y2, "r-", label='Rotated data')
    ax.legend(loc="upper right")
    ax.xaxis_date()
    fig.autofmt_xdate()
    plt.show()

### 1. Reading event list:
The first step is to create a list of events to be pre-processed. This task is performed by using the attribute *add_list_events*

In [ ]:
file_list='events_list.txt'
ev_file_list=local_preprocessing_mtuq.add_list_events(file_list)

The object *ev_file_list* is a list of objects of the class *Earthquake*. Each element of the list (i.e., an object of the class Earthquake) contains the information provided in the file *events_list.txt*. That information can be accessed via the objects using the attributes: 

- or_time: event origin time.
- lat: event latitude.
- lon: longitude.
- depth: depth.
- m: magnitude.
- directory: directory where the waveforms are located.
- traces: seismograms recorded for this event.

At this point, the traces and directory attributes are empty. They will be filled later. 

Let's access one of the attributes:

In [ ]:
#Accessing the attribute or_time
for ev in ev_file_list:
    print(ev.or_time)

### 2. Reading the DATA directory:
In this step the script will search the **DATA** directory for sub-directories for each event to preprocess. The idea is to contrast the data stored in **DATA** with the information gathered in add_list_events.

In [ ]:
events_dir = 'DATA'
ev_dir_list = local_preprocessing_mtuq.add_directory_events(events_dir)

Similarly to *ev_file_list*, this previous step created the object *events_dir*. This is a list that collects objects of the class *Dir_earthquake* whose only attributes are:

- or_time: origin time.
- directory: names of the event directories found in DATA.

For instance, the directory attribute for each element of the list is:

In [ ]:
#Check the attribute directory
for ev_dir in ev_dir_list:
    print(ev_dir.directory)

In this case, we have a unique object unlike for the *ev_file_list* list. This is because in the **DATA** directory there are data for only one earthquake.    

###  3. Compare two lists:

In this step we check *ev_file_list* list against *ev_dir_list* to create a unique list of the class *Earthquake* that only contains events with data in the **DATA** directory. 

In [ ]:
events_dir = 'DATA'
joint_event_list=local_preprocessing_mtuq.merge_lists(ev_file_list,ev_dir_list,events_dir)

The new list *joint_event_list* is similar to *ev_file_list*. However, the objects of the class *Earthquake* in the list *joint_event_list* contain information in the attributes *directory* and *traces* (unlike the empty values in *ev_file_list*).

You can see both attributes by running the following block code. 

In [ ]:
#See traces for each directory in the joint_event_list list 
for ev in joint_event_list:
    print (ev.directory)
    
    for tr in ev.traces:
        print('\t {}'.format(tr))

###  4. Quality control:

This step filters out traces that do not fullfil the following requirements:

- **More than 3 traces per station, channel, and location:** this requirement can be fixed by merging multiple SAC traces for a single station. Ideally, there must be at maximum three traces per station one for each Z, N, and E. 

- **Only one horizontal component:** if only a single horizontal component, either east or north is found, it will be rejected, since one step of the processing consists of rotating to radial and transverse components, 

- **Data start after the origin time:** the trace must start before or at the origin time. In addition, the end time of the trace must be after the origin time. 

- **Existence of a pole and zero SAC file:** each trace must have its corresponding pole an zero file in the pzfiles directory. 


In [ ]:
events_dir = 'DATA'
filter_list_events = local_preprocessing_mtuq.quality_control(joint_event_list,events_dir)

Since all traces passed the quality control *filter_list_events* list is identical to *joint_event_list*.

In [ ]:
#See traces that passed the quality control test 
for ev in filter_list_events:
    print (ev.directory)
    
    for tr in ev.traces:
        print('\t {}'.format(tr))

### 5. First pre-processing:

With the traces that passed the quality control the following processing steps are applied:

- **5.1 Cutting traces and zero padding (if necessary):** the traces are cut to start at the origin time. If traces for a single station have different lengths, then shorter traces are padded with zeros to make all traces have the same lenght.

- **5.2 Detrend traces:** for each trace, [detrend is applied](https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.detrend.html) twice for removing the mean and the linear trend. 

- **5.3 Correcting and completing SAC headers:** for each trace, the [SAC header values](http://www.adc1.iris.edu/files/sac-manual/manual/file_format.html) *depmin, depmax, depmen, evla, evlo, evdp, lovrok (TRUE), lcalda (TRUE),* and *khole* are updated. If the *khole* header value does not exist in the raw trace, then that variable is set in the header as an empty character. In addition, *OMARKER* is created in the SAC header for marking the origin time in the trace. 

- **5.4 Saving pre-processed seismogram in a new location:** the new pre-processed traces with the name format requested by MTUQ are saved in the directory **PROCESSED_DATA** with the same structure as in the **DATA** directory. In this step the *stla*, *stlo* and *knetwk* header values are added via reading the *station_list.txt* file. If the station location is not available in *station_list.txt*, then that trace will not be considered. 

In [ ]:
#Launching the first pre-processing
processed_dir='PROCESSED_DATA'
local_preprocessing_mtuq.preprocessing(joint_event_list,processed_dir,events_dir)

The new SAC files are in the directory **PROCESSED_DATA/20171201023244**. Further pre-processing will be applied to  these SAC files. 

In [ ]:
#Let's see the files in the new directory
! ls PROCESSED_DATA/20171201023244/

Also observe that the header values for the new SAC files are more complete compared to the raw SAC files:

In [ ]:
#See the header of one of the new SAC files
st = read('PROCESSED_DATA/20171201023244/20171201023244.IN.BSRN..BH.z')
print(st[0].stats)

### 6. Correcting for sensor mis-orientation: 

In some cases, seismometers are not oriented to geographic North and hence the horizontal components of the seismograms carry a systematic error. This may cause issues when deviations are large and therefore, a rotation has to be applied to correct the data for sensor misorientation. In the case for Iran, [Braunmiller et al., 2020](https://pubs.geoscienceworld.org/ssa/srl/article/91/3/1660/583389/Sensor-Orientation-of-Iranian-Broadband-Seismic) found those deviations based on P-wave particule motion analysis. 

In this section, a corrective rotation is applied to the horizontal traces (east and north components) if the sensor is  misorientated. The deviation (if exist) is read from the *AZIMUTH* comment in the SAC poles and zero files.

In [ ]:
#Rotating to the true north if it is neccesary 
#First, a  security copy just in case and for educational purposes for comparing the traces before and after the rotation.

processed_dir='PROCESSED_DATA'
! cp -r PROCESSED_DATA BU_PROCESSED_DATA
local_preprocessing_mtuq.rotate_true_north(processed_dir)

In this example, the three stations have a deviation from true north: TNSJ (23°), BSRN (-3°) and NHDN (19°). Therefore, all horizontal traces were rotated and the SAC files overwritten. 

Now we are going to use the plot function defined earlier in this notebook for comparing the raw waveforms and the rotated ones.

#### NOTE: the raw unrotated data is backed-up in BU_PROCESSED_DATA. This backup could be useful in case that the rotation is applied more than once by mistake. 

In [ ]:
#Let´s compare waveforms
st = read('PROCESSED_DATA/20171201023244/20171201023244.IN.BSRN..BH.e')
st += read('BU_PROCESSED_DATA/20171201023244/20171201023244.IN.BSRN..BH.e')
#This zoom paremeter cut a percentage (0 to 1)from the begining and the end of the trace. 
#In this case we omit the 20% of the onset and 90% from the end.
zoom = [0.02,0.9]
plot_event(st[0],st[1],zoom)

Station BSRN has a tiny deviation from true North (-3°), so it is expected that the seismogram after the rotation is almost identical to the unrotated one.  

In [ ]:
# Repeat the process for a station with a larger deviation:
st = read('PROCESSED_DATA/20171201023244/20171201023244.IR.TNSJ..BH.n')
st += read('BU_PROCESSED_DATA/20171201023244/20171201023244.IR.TNSJ..BH.n')
zoom = [0.10,0.6]
plot_event(st[0],st[1],zoom)

Station TNSJ has a large deviation (23°) and the difference in the amplitudes between the rotated and unrotated data is consistent with this. A similar result can be found for NHDN (19°):

In [ ]:
st = read('PROCESSED_DATA/20171201023244/20171201023244.IR.NHDN..BH.n')
st += read('BU_PROCESSED_DATA/20171201023244/20171201023244.IR.NHDN..BH.n')
zoom = [0.13,0.5]
plot_event(st[0],st[1],zoom)

#### NOTE: An important sanity check is to confirm that the vertical components remain unmodified since the rotation has to be applied only to the horizontal ones. 

In [ ]:
#Sanity Check: vertical components should be identical
st = read('PROCESSED_DATA/20171201023244/20171201023244.IR.NHDN..BH.z')
st += read('BU_PROCESSED_DATA/20171201023244/20171201023244.IR.NHDN..BH.z')
zoom = [0.13,0.5]
plot_event(st[0],st[1],zoom)

### 7. Removing the instrument response:

#### 7.1 Making RESP files:
For [removing the instrument response in ObsPy](https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.remove_response.html) it is necessary to create an inventory object using the [read_inventory](https://docs.obspy.org/packages/autogen/obspy.core.inventory.inventory.read_inventory.html) method. However, the SAC pole and zero files are not included among the options of input parameters of such method. The closest SAC format is the [RESP file](https://ds.iris.edu/ds/nodes/dmc/data/formats/resp/) format. Therefore, in the next step we read the pole and zero SAC files to create RESP files for the sole purpose of removing the instrument response. 

#### NOTE: 
Because of their nature, RESP files are more comprenhensive than the pole and zero SAC files. Therefore, if RESP files are available a better approach is to use those RESP files to [create pole and zero files](https://docs.obspy.org/packages/autogen/obspy.io.sac.sacpz._write_sacpz.html). The opposite procedure (like in this example) creates RESP files that contain some dummy values. The RESP files created in this step are only useful for removing the instrument response if the poles, zeroes, and constant information is correct; in this approach only the seismic sensor response is removed but any digitizer related system response will not be removed.

For example, in the dummy RESP file the *A0 normalization factor* is filled with the *CONSTANT* value taken from the SAC pole and zero file. This is innacurate since the *CONSTANT* is equal to the product of the *A0 normalization factor*, and the *Sensitivity* (which is obtained by the product of the different gain values) in the RESP file. In this (unelegant but functional) approach, the *Sensitivity* and the gains in the RESP file are defined as 1.0 so the final product between the *A0 normalization factor* and *Sensitivity* is equal to the *CONSTANT*. 

In case that you do not have full access to sensor information for creating a RESP file or you suspect incorrect information in existing RESP files asides from poles, zeroes and constant, then the method presented here is a viable option.  

In [ ]:
#Make the resp files for removing the instrumental response
processed_dir = 'PROCESSED_DATA'
local_preprocessing_mtuq.paz2resp(processed_dir)

The RESP files are createad in a directory named **RESP_FILES**. Be aware those files are only valid for removing the sensor (instrument) response.

In [ ]:
#Check one of the RESP files:
! cat RESP_FILES/RESP.IR.NHDN..BHZ

#### 7.2 Use the dummy RESP files for removing the instrument response:

The next block of code uses the RESP files and removes the instrumental response for the SAC files saved in PROCESSED_DATA.

In [ ]:
#Remove the instrumental response for all the files stored in PROCESSED_DATA
local_preprocessing_mtuq.remove_instrumental_response_dir(processed_dir)

### 8. Rotate to the radial and transverse:

In this step  are created the radial and transverse components to be used in MTUQ.

In [ ]:
#Rotating to the Radial and Transverse
processed_dir = 'PROCESSED_DATA/'
local_preprocessing_mtuq.rotate_radial_transverse_dir(processed_dir)

Now, each station must have 5 traces: vertical, East, North, radial and transverse.

In [ ]:
# Checking the new traces:
! ls PROCESSED_DATA/20171201023244/*

### 9. Padding zeros:

In some cases for MTUQ, depending on the maximum time-shifts allowed and the lenght of the seismogram, there might not be enough samples before the origin time and zeros should be added. You can use either the following block of code or the function included in MTUQ to do this.

For this example, 60 s of zeros will be added.

#### NOTE: the following method operates on single event directories rather than on all  events in PROCESSED_DATA.

In [ ]:
#Padd zeros
processed_dir = 'PROCESSED_DATA'
event = '{}/20171201023244'.format(processed_dir)
time = 60
local_preprocessing_mtuq.padd_zeros(event,time)

### 10. Scaling the amplitudes:
The unit of the seismogram after removing the instrument response is m/s. However, we use MTUQ in centimeters (cm) for consistency with the [MTUQ seismic moment units](https://github.com/uafgeotools/mtuq/blob/260b827fe8f4934986efd6172b9cc45eecc34318/mtuq/greens_tensor/FK.py#L19) for the Green's Functions. 

Therefore, in this example, the following block of code multiplies the seismogram by 100 to convert from m/s to cm/s. As for the previous step, this method operates on single event directories. 

In [ ]:
#Scale Amplitude
processed_dir = 'PROCESSED_DATA'
event = '{}/20171201023244'.format(processed_dir)
scale = 100
local_preprocessing_mtuq.scale_amplitude(event,scale)

### 11. Making weights.dat files:

The last step before running MTUQ is to make the parameter input file read by MTUQ. 
The user must define whether body and/or surface waves will be included and which components will be used. 
This is done by defining the components object in the following block of code. 

For example, if both body (vertical and radial) and surface (vertical, radial and transverse) waves with all  components will be included for the moment tensor estimation, then the components object has to be set up as:

components = '1   1   1   1   1'

If only the vertical component for body waves and transverse and radial components of the surface waves are going to be used, the components object will look like:

components = '1   0   1   0   1'

#### NOTE: The configuration of the *components* object will be applied to all events in *PROCESSED_DATA*

In [ ]:
#Writting weights.dat files for using all componentes of body and surface waves
processed_dir = 'PROCESSED_DATA'
components = '1 1 1 1 1'
local_preprocessing_mtuq.write_weight_all(processed_dir,components)

In [ ]:
#Check the weights.dat file in 20171201023244
! cat PROCESSED_DATA/20171201023244/weights.dat

### Now you are ready to run MTUQ with your local DATA!

### 12. Running the MTUQ example:

In the directory **run_MTUQ** you will find a python script for running a double couple constrained moment tensor estimation in MTUQ (*FK_GFs_GridSearch_DoubleCouple_options.py*). For this example, we also provide the Green's Functions pre-calculated with the FK-method for a regional 1D velocity model in Iran for 6 km source depth (stored in directory *greens/ir/ir_6*).

In [ ]:
#Use this code of block once. Is for backing-up the main directory and going back to it when neccesary
import os
main_dir = os.getcwd()

The first step is to copy the Pre-processed seismograms to the *run_MTUQ* directory:

In [ ]:
! cp -r PROCESSED_DATA/20171201023244 run_MTUQ/

Second, take a look to the Green Functions to use for calculating the synthetic seismograms:

In [ ]:
#let's check the Green Functions (GF's)
! ls run_MTUQ/greens/ir/ir_6/

The Green's Functions provided for this example are calculated for a source at 6km depth and distances of 216, 264, and 366 km between the epicenter and the stations. 

#### NOTE: 
The distances between epicenter and stations are integer values in MTUQ. Therefore MTUQ uses the numpy ceil method for rounding distances, which in some cases may create [conflicts](https://github.com/uafgeotools/mtuq/issues/173) for matching the distance calculated from the SAC files and the corresponding set of Green's Functions. This is because the round method could be used for making the weights.dat files. A patch to circumvent discrepancies in both methods for rounding the distances is to calculate Green's Functions for a distance (d) and two more distances (d+1, d-1). For example, for station TNSJ the correspondent Green's Functions are 366.grn.* but we also provided the Green's Functions 365.grn.* and 367.grn.*. 

Now, move to the **run_MTUQ** directory for running the *FK_GFs_GridSearch_DoubleCouple_options.py* script:

In [ ]:
os.chdir(main_dir)
os.chdir('run_MTUQ')

#### 12.1 Launching the MTUQ grid-search:
The *FK_GFs_GridSearch_DoubleCouple_options.py* script performs a double couple grid-search. Once the module is loaded:

In [ ]:
#Loading the module FK_GFs_GridSearch_DoubleCouple_options
import FK_GFs_GridSearch_DoubleCouple_options
import importlib
importlib.reload(FK_GFs_GridSearch_DoubleCouple_options)

It is possible to use the method launch_gs for starting the grid-search. Use the doscstring method to find out about the input options:

In [ ]:
print (FK_GFs_GridSearch_DoubleCouple_options.launch_gs.__doc__)

In the following block of code, the method launch_gs is called using the input parameters for the event and the corresponding pre-processed waveforms for this example.

In [ ]:
FK_GFs_GridSearch_DoubleCouple_options.launch_gs('20171201023244',30.734,57.39,6000.0,6.0,'2017-12-01T02:32:44.000000Z',30,'3-15','15-33',25,150)

The final step is to open the figure *20171201023244DC_waveforms.png* that summarizes the result of the moment tensor estimation - in this case we had constrained the source to be DC.

In [ ]:
from IPython.display import Image, display
display(Image(filename='20171201023244DC_waveforms.png'))

# The END